## PySpark Test script to collate blobs of JSON together

This is in order to collate together separate JSON scripts that are accepted in and stored via the inbound API
the inbound API will store json locally, and clean itself on a scheduled basis
The point of this exercise is to gather inputs A B and C together, combine them, and store them in a separate location

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [15]:
spark = SparkSession.Builder().master('local[*]')\
    .appName("inboundcollation")\
    .getOrCreate()

In [16]:
inboundDir = '../data/aggregate/'

jsonSchema = StructType([
    StructField("title", StringType(), False),
    StructField("author", StringType(), True),
    StructField("date_published", StringType(), True),
    StructField("lead_image_url", StringType(), True),
    StructField("content", StringType(), False),
    StructField("next_page_url", StringType(), True),
    StructField("url", StringType(), False),
    StructField("domain", StringType(), True),
    StructField("excerpt", StringType(), True),
    StructField("word_count", IntegerType(), False),
    StructField("direction", StringType(), True),
    StructField("total_pages", IntegerType(), True),
    StructField("rendered_pages", IntegerType(), True),
    StructField("keywords", ArrayType(StringType()), True)
])


In [17]:
frame = open('../data/aggregate/').read()

df_app = spark.read.schema(jsonSchema).json("../data/aggregate/", multiLine=True)

Py4JJavaError: An error occurred while calling o150.json.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1215)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1420)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(Trav

In [6]:
df_app.show()

+--------------------+----------+--------------------+--------------------+--------------------+-------------+--------------------+---------------+--------------------+----------+---------+-----------+--------------+----------------+
|               title|    author|      date_published|      lead_image_url|             content|next_page_url|                 url|         domain|             excerpt|word_count|direction|total_pages|rendered_pages|        keywords|
+--------------------+----------+--------------------+--------------------+--------------------+-------------+--------------------+---------------+--------------------+----------+---------+-----------+--------------+----------------+
|Application Perfo...|Matt Makai|2019-05-21T16:21:...|https://www.fulls...|\nAmazon Web Serv...|         null|https://www.fulls...|fullstackpython|Learn how to use ...|      1113|      ltr|          1|             1|[python, lambda]|
+--------------------+----------+--------------------+----------

In [ ]:
# UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows this occurs because winUtils isn't installed - winutils simulates a Hadoop installation, look into adding this and setting your %HADOOP_HOME path to this
# this is the only way to be able to run pySpark jobs, which gives me pause for the project
# potential resolutions may be to do this via a container